In [1]:
import os
import sys
import pandas as pd
import numpy as np
import lightfm
import scipy.sparse as sps
import scipy.sparse.linalg as splinalg

In [2]:
threads=10

In [3]:
i=40

In [4]:
print("running batch %d" % i)
batch = pd.read_csv("batches/batch_%d_train.dat" % i)
test_users = pd.read_csv("batches/batch_%d_test.dat" % i)

running batch 40


In [5]:
model = lightfm.LightFM(
    loss='warp',
    no_components=10,
    learning_rate=0.004,
    learning_schedule="adadelta"
)

In [6]:
maxover = batch.groupby('user').item.count().max()

In [7]:
topk = 100

def get_ranklists(model, users, items, test):
    import concurrent.futures
    executor = concurrent.futures.ThreadPoolExecutor(threads)
    def predu(i):
        scores = model.predict(
            i,
            items,
            num_threads=1
        )
        return items[np.argsort(scores)[-(topk+maxover):][::-1]]
    
    preds = list(executor.map(predu, users))
    lists = pd.DataFrame({
        'user': np.repeat(users, topk+maxover),
        'item': np.ndarray.flatten(np.array(preds)),
        'pos': np.tile(np.arange(topk+maxover)+1, len(users))
    })
    return lists

In [8]:
uim_train = sps.coo_matrix((np.ones(len(batch)),tuple(zip(*batch[['user','item']].values))))

In [9]:
model = model.fit_partial(
    uim_train,
    epochs=30,
    num_threads=threads,
    verbose=True
)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [10]:
real_test_users = test_users.user[test_users.user.isin(batch.user.unique())].values
ranklists = get_ranklists(model, real_test_users, batch.item.unique(), None)

In [11]:
# filtering seen items
ranklists_j = ranklists.join(batch.set_index(['user','item'])['score'], on=['user','item'])
ranklists_j_new = ranklists_j[ranklists_j['score'].isnull()].copy()
ranklists_j_new.rename(columns={'pos':'oldpos'}, inplace=True)
ranklists_j_new.sort_values('oldpos', inplace=True)
ranklists_j_new['pos'] = 1
ranklists_j_new['pos'] = ranklists_j_new.groupby('user').pos.transform(np.cumsum)

In [12]:
ranklists_j_new[ranklists_j_new.pos <= 100][['user', 'item', 'pos']].to_csv('batches/batch_%d_predictions.dat' % i, sep=' ', header=False)